In [0]:
#Welcome to your new notebook
#Establishing connection to the storage account
service_credential = dbutils.secrets.get(scope="<your-scope-name>",key="<your-service-credential-key>")

 
spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")

spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")

spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<application-id>")

spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", service_credential)

spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory-id>/oauth2/token")

In [0]:
coaches = spark.read.format("csv").option("header", "true").load("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/raw_data/coaches.csv")

display(coaches)

code,current,name,gender,function,category,country_code,country,country_long,disciplines,events,birth_date
1533246,True,PEDRERO Ofelia,Female,Coach,C,MEX,Mexico,Mexico,Artistic Swimming,Team,1988-03-28
1535775,True,RADHI SHENAISHIL,Male,Head Coach,C,IRQ,Iraq,Iraq,Football,Men,1965-07-01
1536055,True,AFLAKIKHAMSEH Majid,Male,Coach,C,IRI,IR Iran,Islamic Republic of Iran,Taekwondo,null,1973-08-26
1536059,True,YOUSEFY Mehrdad,Male,Coach,C,IRI,IR Iran,Islamic Republic of Iran,Taekwondo,null,1972-06-12
1536060,True,MADDAH Minoo,Female,Coach,C,IRI,IR Iran,Islamic Republic of Iran,Taekwondo,null,1976-05-17
1536328,True,LOFTUS Adriana,Female,Coach,C,MEX,Mexico,Mexico,Artistic Swimming,Team,1958-06-28
1538313,True,FERRARA Fernando,Male,Head Coach,C,ARG,Argentina,Argentina,Hockey,null,1968-07-24
1538315,True,GULLA Alejandra,Female,Assistant Coach,C,ARG,Argentina,Argentina,Hockey,Women,1977-07-04
1538317,True,CAPURRO Santiago,Male,Assistant Coach,C,ARG,Argentina,Argentina,Hockey,null,1975-04-08
1538745,True,RONCONI Mariano,Male,Head Coach,C,ARG,Argentina,Argentina,Hockey,null,null


In [0]:
#Loading the rest of the data
#athletes
athletes = spark.read.format("csv").option("header", "true").load("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/raw_data/athletes.csv")


#medals
medals = spark.read.format("csv").option("header", "true").load("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/raw_data/medals.csv")

#teams
teams = spark.read.format("csv").option("header", "true").load("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/raw_data/teams.csv")


In [0]:
#checking the schema
medals.printSchema()

root
 |-- medal_type: string (nullable = true)
 |-- medal_code: string (nullable = true)
 |-- medal_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- event: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- url_event: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.filter(medals["medal_type"] == "Gold Medal").groupBy("Country").count().orderBy("count", ascending=False)
display(top_gold_medal_countries)

Country,count
China,40
United States,40
Japan,20
Australia,18
France,16
Netherlands,15
Great Britain,14
Korea,13
Italy,12
Germany,12


In [0]:
top_gold_medal_countries.printSchema()

root
 |-- Country: string (nullable = true)
 |-- count: long (nullable = false)



In [0]:
# Save the teams DataFrame to a folder
teams.write.format("csv").option("header", "true").mode("overwrite").save("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/transformed_data/teams")

top_gold_medal_countries.write.format("csv").option("header", "true").mode("overwrite").save("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/transformed_data/top_gold_medal_countries")

athletes.write.format("csv").option("header", "true").mode("overwrite").save("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/transformed_data/athletes")

medals.write.format("csv").option("header", "true").mode("overwrite").save("abfss://tokyoolympicscontainer@tokyoolympicstore2.dfs.core.windows.net/transformed_data/medals")
